In [3]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum
import matplotlib.pyplot as plt



class Production_data:
    def __init__(self, number_of_periods, number_of_items, demand_forecast, production_cost, holding_cost, setup_cost, item_requirements , capacity):
        self.T=number_of_periods
        self.items=number_of_items
        self.demand_forecast=np.array(demand_forecast)
        self.production_cost=np.array(production_cost)
        self.holding_cost=np.array(holding_cost)
        self.setup_cost=np.array(setup_cost)
        self.item_requirements=np.array(item_requirements)
        self.capacity=np.array(capacity)

file_name = "CLSP+ST-instances Data-R.xlsx"
#file_name = "prova2.xlsx"

xls = pd.ExcelFile(file_name)  # Read the whole file

tables_keywords = ["Demand Forecast:", "Production Cost", "Holding Cost", "Setup Cost", "UnitsOfCapacity", "Capacity"]


In [10]:


def read_data(xls, sheet_name):
    tables_dict = {}
    df = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Define which column to check for each keyword
    columns_to_check = {
        "Demand Forecast:": 0,
        "Production Cost": 0,
        "Holding Cost": 0,
        "Setup Cost": 0,
        "UnitsOfCapacity": 1,  # Check the second column for this keyword
        "Capacity": 0
    }
    
    # Iterate through the keywords to find each table
    for keyword in tables_keywords:
        column_idx = columns_to_check.get(keyword, 0)
        
        # Check in the specified column for the keyword
        match = df[df.iloc[:, column_idx].astype(str).str.contains(keyword, na=False)]
        
        if not match.empty:
            table_start_row = match.index[0] + 1
            
            # Find the end of the current table (next keyword or empty rows)
            end_row = None
            for next_keyword in tables_keywords:
                if next_keyword != keyword:
                    next_column_idx = columns_to_check.get(next_keyword, 0)
                    next_match = df.loc[table_start_row:][df.loc[table_start_row:].iloc[:, next_column_idx].astype(str).str.contains(next_keyword, na=False)]
                    if not next_match.empty:
                        potential_end = next_match.index[0]
                        if end_row is None or potential_end < end_row:
                            end_row = potential_end
            
            # If no next keyword found, look for empty rows
            if end_row is None:
                for i in range(table_start_row, len(df)):
                    # Check if row is empty or contains only NaN values
                    if df.iloc[i].isna().all():
                        end_row = i
                        break
            
            # If still no end found, use the end of the dataframe
            if end_row is None:
                end_row = len(df)
            
            # Extract the table
            table_df = df.iloc[table_start_row:end_row]
            
            # Remove completely empty rows
            table_df = table_df.dropna(how='all')
            
            # Remove completely empty columns
            table_df = table_df.dropna(axis=1, how='all')
            
            # Remove any remaining NaN values by filling with 0
            table_df = table_df.fillna(0)
            
            # Convert to numpy array
            table = table_df.to_numpy()
            tables_dict[keyword] = table
    
    # Create an instance of Production_data class
    production_data = Production_data(
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[1]-1,
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[0]-1,  
        tables_dict.get("Demand Forecast:", np.zeros((1,1))),  
        tables_dict.get("Production Cost", np.zeros((1,1))),  
        tables_dict.get("Holding Cost", np.zeros((1,1))),  
        tables_dict.get("Setup Cost", np.zeros((1,1))),  
        tables_dict.get("UnitsOfCapacity", np.zeros((1,1))),  
        tables_dict.get("Capacity", np.zeros((1,1)))

    )
    
    production_data.capacity = np.vstack([
    np.zeros((1, production_data.capacity.shape[1]), dtype=production_data.capacity.dtype),production_data.capacity])

    production_data.item_requirements = np.vstack([
    np.zeros((1, production_data.item_requirements.shape[1]), dtype=production_data.item_requirements.dtype),production_data.item_requirements
])
    return production_data 

#data=read_data(xls, "Data-20-12 (1)")

#print(data.T)
#print(data.items)

#print(data.demand_forecast)
#print(data.holding_cost)
#print(data.setup_cost)
#print(data.capacity)
#print(data.production_cost)

#print(data.capacity[0, 1])
#print(data.capacity[12, 1])
#print(data.item_requirements)

sheet_list=["Data-20-24 (1)"]
#["Data-20-12 (2)"]#, "Data-20-12 (2)", "Data-20-24 (1)", "Data-20-24 (2)" , "Data-100-24 (1)", "Data-100-24 (2)",
            

#sheet_list=["Sheet1"]





In [11]:
#yit is the complicating variable, to fix, we fix it so then the problem becomes a simple LP 
#ok is gurobi the one that chooses the values for y
#create dual and if dual feasible we add cut


import time

def Benders_decomposition(data, sheet_name, max_iterations):
    # Initialize master problem
    master = gp.Model("Benders Master Problem")
    
    T = data.T
    O = data.items


    start_time = time.time()
    time_limit = 30 * 60  # 30 minutes
    LBs = []
    UBs = []

    
    # Master problem variables
    y = master.addVars(O+1, T+1, vtype=GRB.BINARY, name="y")  # setup decisions
    theta = master.addVar(vtype=GRB.CONTINUOUS, lb=0, name="theta")  # variable for optimality cuts

    # Master problem objective
    master.setObjective(
        quicksum(quicksum(data.setup_cost[i, t] * y[i, t] for i in range(1, O+1)) for t in range(1, T+1)) +
        theta,
        GRB.MINIMIZE
        
    )
    # Production limit constraints contribution
    feas_cut_expr_prod = quicksum(
                quicksum( (sum(data.demand_forecast[i, q] for q in range(t, T+1)) * y[i, t])
                        for i in range(1, O+1)) #alright
                for t in range(1, T+1) 
        )

    # Flow constraints contribution
    feas_cut_expr_flow = quicksum( quicksum(data.demand_forecast[i,t] for i in range(1,O+1)) for t in range(1,T+1))

    master.addConstr(feas_cut_expr_flow<=feas_cut_expr_prod, name="initial cut")


    # Get initial y values from MILP solution
    y_vals = np.zeros((O+1,T+1))

    for i in range(1, O+1):
        y_vals[i, 1]=1


    UB=float("inf")
    LB=float("-inf")

    masterobj = []
    fesibilitysubobj = []
    
    for iteration in range(1, max_iterations):

        print(f"\nIteration {iteration}")
        
        # Get Farkas certificates from feasibility subproblem
        capacity_duals, prod_limit_duals, flow_duals, somma = solve_feasibility_subproblem(y_vals, data)
    
        if somma > 1e-6:  # If artificial variables are used, add feasibility cut
            #subproblem infeasible, need to add feasibility cut
            print("Subproblem is infeasible → Adding feasibility cut")
                
            # Capacity constraints contribution
            feas_cut_expr_cap = quicksum(
                +capacity_duals[t] * (data.capacity[t, 1] - quicksum(data.item_requirements[i, 2] * y[i, t]
                for i in range(1, O+1))) #alright
                for t in range(1, T+1)
            )

            # Production limit constraints contribution
            feas_cut_expr_prod = quicksum(
                quicksum(+prod_limit_duals[i, t] * 
                        (sum(data.demand_forecast[i, q] for q in range(t, T+1)) * y[i, t])
                        for i in range(1, O+1)) #alright
                for t in range(1, T+1)
            )

            # Flow constraints contribution
            feas_cut_expr_flow = quicksum( quicksum(+flow_duals[i, t]*data.demand_forecast[i,t] for i in range(1,O+1)) for t in range(1,T+1)
                    
            )
            # Add combined feasibility cut 
            master.addConstr(
                feas_cut_expr_cap + feas_cut_expr_prod + feas_cut_expr_flow<= 0,
                name=f"feasibility_cut_{iteration}"
            )
                
            print(f"Added feasibility cut in iteration {iteration}")

        else:
            print("Primal subproblem is optimal → Adding optimality cut")

            subprob, x_vals, s_vals = create_and_solve_subprob(y_vals, data)

            # Get dual values from optimal subproblem
            capacity_duals = {}
            prod_limit_duals = {}
            
            for t in range(1, T+1):
                constr = subprob.getConstrByName(f"capacity_{t}")
                capacity_duals[t] = constr.Pi
                
            for t in range(1, T+1):
                for i in range(1, O+1):
                    constr = subprob.getConstrByName(f"prod_limit_{i}_{t}")
                    prod_limit_duals[i,t] = constr.Pi

            # Add optimality cut
            opt_cut_expr = quicksum(
                +capacity_duals[t] * (data.capacity[t, 1] - quicksum(data.item_requirements[i, 2] * y[i, t]
                for i in range(1, O+1)))
                for t in range(1, T+1)
            ) + quicksum(
                quicksum(+prod_limit_duals[i, t] * 
                        (sum(data.demand_forecast[i, q] for q in range(t, T+1)) * y[i, t])
                        for i in range(1, O+1))
                for t in range(1, T+1)
            )
            + quicksum( quicksum(+flow_duals[i, t]*data.demand_forecast[i,t] for i in range(1,O+1)) for t in range(1,T+1))

            
            
            master.addConstr(theta >= opt_cut_expr, name=f"optimality_cut_{iteration}")
            print(f"Added optimality cut in iteration {iteration}")
            
            
        # Solve updated master problem

        remaining_time = max(0, time_limit - (time.time() - start_time))
        master.setParam("TimeLimit", remaining_time)
        master.optimize()

        
        if master.status == GRB.OPTIMAL:
            # Update y values
            for t in range(1, T+1):
                for i in range(1, O+1):
                    y_vals[i][t] = y[i, t].X

        elapsed_time = time.time() - start_time
        if elapsed_time >= time_limit:
            print("Time limit reached. Stopping early.")
            break

        LB=master.ObjVal
        LBs.append(LB)

        if somma<=1e-6 and subprob.Status == GRB.OPTIMAL:
            UB=subprob.ObjVal
        
        if UB-LB<1:
            print("Converged!!!!!")
            print(f"UB: {master.ObjVal}")
            break


    # After the loop
    plt.figure(figsize=(10, 6))
    plt.plot(LBs, label='Lower Bound (LB)', marker='o')
    plt.plot(UBs, label='Upper Bound (UB)', marker='x')
    plt.xlabel("Iteration", fontsize=14)
    plt.ylabel("Objective Value", fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"LB_UB_{sheet_name}.png")
    plt.show()


    
            
            
    return y_vals,

def create_and_solve_subprob(y_vals, data): #THIS IS P1, THE SUBPROBLEM
    #IT IS HIGHLY POSSBILE THAT IT IS UNFEASIBLE FOR THE Ys PROVIDED

    #THIS IS ALRIGHT

    T=data.T
    O=data.items

    #now we need to do the subproblem
    sub=gp.Model("Subproblem")

    s=sub.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s") #amount of inv of item i in period t
    x=sub.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x") #amount produced of item i in period t


    #this is P1
    sub.setObjective(
        quicksum(quicksum( data.production_cost[i, t] * x[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.holding_cost[i, t] * s[i, t] for i in range(1, O+1) ) for t in range(1, T+1)),
    GRB.MINIMIZE
    )  

    #constraint 2
    for t in range(1, T+1):
        for i in range(1, O+1):
            sub.addConstr( s[i, t-1] + x[i, t] - s[i, t] == data.demand_forecast[i, t], 
                         name=f"flow_{i}_{t}")

    #constraint 3
    for t in range(1, T+1):
        sub.addConstr(quicksum(x[i, t] * data.item_requirements[i, 1] + data.item_requirements[i,2] * y_vals[i][t]   for i in range(1, O+1)) <= data.capacity[t, 1], 
             name=f"capacity_{t}")
            
    #constraint 4
    for t in range(1, T+1):
        for i in range(1, O+1):
            sub.addConstr(x[i, t]-(quicksum(data.demand_forecast[i, q] 
                                          for q in range(t, T+1))) * y_vals[i][t] <= 0, 
                         name=f"prod_limit_{i}_{t}")

    #constraint 7
    for i in range(1, O+1):
        sub.addConstr(s[i, 0] == 0, name=f"init_inv_{i}")
        sub.addConstr(s[i, T] == 0, name=f"fin_inv_{i}")

    
    sub.optimize()
        

    return sub, x, s


def solve_feasibility_subproblem(y_vals, data):
    T = data.T
    O = data.items

    feas = gp.Model("Feasibility_Subproblem")
    # Variables
    s = feas.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s")
    x = feas.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")

    # Artificial variables for each constraint
    a_flow = feas.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="a_flow")
    a_cap = feas.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="a_cap")
    a_prod = feas.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="a_prod")
    a_init = feas.addVars(O+1, vtype=GRB.CONTINUOUS, lb=0, name="a_init")
    a_final = feas.addVars(O+1, vtype=GRB.CONTINUOUS, lb=0, name="a_final")
    
    # Objective: minimize sum of artificial variables
    feas.setObjective(
        quicksum(a_flow[i, t] for i in range(1, O+1) for t in range(1, T+1)) +
        quicksum(a_cap[t] for t in range(1, T+1)) +
        quicksum(a_prod[i, t] for i in range(1, O+1) for t in range(1, T+1)) +
        quicksum(a_init[i] + a_final[i] for i in range(1, O+1)),
        GRB.MINIMIZE
    )

    # Flow constraints with artificial variables
    for t in range(1, T+1):
        for i in range(1, O+1):
            feas.addConstr(
                s[i, t-1] + x[i, t] - s[i, t] + a_flow[i, t] == data.demand_forecast[i, t],
                name=f"flow_{i}_{t}"
            )

    # Capacity constraints
    for t in range(1, T+1):
        feas.addConstr(
            quicksum(x[i, t] * data.item_requirements[i, 1] + data.item_requirements[i, 2] * y_vals[i][t] for i in range(1, O+1)) +
            a_cap[t] <= data.capacity[t, 1],
            name=f"capacity_{t}"
        )

    # Production limit constraints
    for t in range(1, T+1):
        for i in range(1, O+1):
            rhs = sum(data.demand_forecast[i, q] for q in range(t, T+1)) * y_vals[i][t]
            feas.addConstr(
                x[i, t] - rhs + a_prod[i, t] <= 0,
                name=f"prod_limit_{i}_{t}"
            )

    # Inventory constraints
    for i in range(1, O+1):
        feas.addConstr(s[i, 0] + a_init[i] == 0, name=f"init_inv_{i}")
        feas.addConstr(s[i, T] + a_final[i] == 0, name=f"fin_inv_{i}")

    feas.optimize()

    print(f"Feasibility subproblem objective value: {feas.ObjVal}")



    # Get Farkas duals from relaxed constraints
    capacity_duals = {}
    prod_limit_duals = {}
    flow_duals = {}
    
    # Get duals for capacity constraints
    for t in range(1, T+1):
            constr = feas.getConstrByName(f"capacity_{t}")
            capacity_duals[t] = constr.Pi
            #print(f"capacity duals: {capacity_duals[t]}")

        
    # Get duals for production limit constraints
    for t in range(1, T+1):
        for i in range(1, O+1):
            constr = feas.getConstrByName(f"prod_limit_{i}_{t}")
            prod_limit_duals[i,t] = constr.Pi
            #print(f"prod limit duals: {prod_limit_duals[i, t]}")
            
            
    # Get duals for flow constraints
    for t in range(1, T+1):
        for i in range(1, O+1):
            constr = feas.getConstrByName(f"flow_{i}_{t}")
            flow_duals[i,t] = constr.Pi
            #print(f"flow duals: {flow_duals[i, t]}")
        
    # Calculate sum of artificial variables for detecting infeasibility
    flow_artificial=sum(sum(a_flow[i, t].X for i in range(1,O+1)) for t in range(1, T+1))
    #print(f"flow artificial variables sum: {flow_artificial}")
    capacity_artificial=sum(a_cap[t].X for t in range(1, T+1))
    #print(f"capacity artificial variables sum: {capacity_artificial}")
    prod_artificial=sum(sum(a_prod[i, t].X for i in range(1,O+1)) for t in range(1, T+1))
    #print(f"prod artificial variables sum: {prod_artificial}")
    

    summ = flow_artificial+capacity_artificial+prod_artificial 

    return capacity_duals, prod_limit_duals, flow_duals, summ
    



    

    



In [12]:
for sheet_name in sheet_list:
    data=read_data(xls, sheet_name)
    Benders_decomposition(data, sheet_name, 200)


Iteration 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



C:\Users\ricca\AppData\Local\Temp\ipykernel_12144\2548460570.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_12144\2548460570.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_12144\2548460570.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior

GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information